# Copolymer Simulation

Simulate a polymer with repeating A-B units.

### Setup

Load required packages and specify root directory of simulator package.

**Run this code block only once.**

In [1]:
# Built-in modules
import os
import sys
import pprint
import multiprocessing as mp

# Insert package root to system path
cwd = os.getcwd()
parent_dir = cwd + "/../.."
sys.path.insert(1, parent_dir)

print("Directory containing the notebook:")
print(cwd)

Directory containing the notebook:
/Users/jwakim/Documents/Wakim_Research/peaking_sims/AAB/doc/tools


In [4]:
# External modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Package modules
import chromo.mc as mc
import chromo.mc.move_funcs as mv
from chromo.polymers import SSWLC
import chromo.binders
from chromo.fields import UniformDensityField
import chromo.mc.mc_controller as ctrl
from chromo.util.reproducibility import get_unique_subfolder_name
import chromo.util.poly_stat as ps

ModuleNotFoundError: No module named 'chromo.mc.mc_sim'

In [3]:
# Change working directory to package root
os.chdir(parent_dir)
print("Root Directory of Package: ")
print(os.getcwd())

Root Directory of Package: 
/Users/jwakim/Documents/Wakim_Research/peaking_sims/AAB


### Specify Chain Lengths

For this example, we will specify arbitrary chains that contains 40 repeating units.

In [6]:
num_repeats = 40

### Specify pattern of bead types

We will assign the `A` block to a monomer state of 1 and the `B` block to a monomer state of 0.

In [ ]:
state_pattern = np.array([0, 1])

A_states = np.tile(state_pattern, num_repeats)
B_states = (A_states + 1) % 2
states = np.column_stack((A_states, B_states))

bead_length = 0.148
n_polymers = 10

### Define Physical Properties of Bead Types

In [9]:
interaction_length = 1
AA_interaction_strength = 0
BB_interaction_strength = 0
AB_interaction_strength = 1


A_monomer = chromo.binders.ReaderProtein(
    name="A",
    sites_per_bead=1,
    bind_energy_mod=0,                                                  # We will not model binding/unbinding
    bind_energy_no_mod=0,
    interaction_energy=AA_interaction_strength,
    chemical_potential=0,                                               # No effect (not modeling binding/unbinding)
    interaction_radius=interaction_length,                              # Interaction length scale
    cross_talk_interaction_energy={"B": AB_interaction_strength}        # A-B interaction
)
B_monomer = chromo.binders.ReaderProtein(
    name="B",
    sites_per_bead=1,
    bind_energy_mod=0,                                                  # We will not model binding/unbinding
    bind_energy_no_mod=0,
    interaction_energy=BB_interaction_strength,
    chemical_potential=0,                                               # No effect (not modeling binding/unbinding)
    interaction_radius=interaction_length,                              # Interaction length scale
)
monomers = chromo.binders.make_binder_collection(
    [A_monomer, B_monomer]
)

NameError: name 'chromo' is not defined

### Specify Polymer Characteristic Properties

In [7]:
bead_length = 1
lp = 1
n_polymers = 10

# TODO: If you want to change the output formatting so that all configurations are appended to the same file, you
#       can do this in chromo.mc.__init__.py

### Instantiate Polymers

In [ ]:
polymers = []
for i in range(n_polymers):
    polymer = SSWLC.gaussian_walk_polymer(
        name = f"poly_{i+1}",
        num_beads = num_repeats * len(state_pattern),
        lp = lp,
        bead_length = bead_length,
        states = states.copy(),
        binder_names = np.array(["A_monomer", "B_monomer"]),
        chemical_mods = states.copy(),
        chemical_mod_names = np.array(["A_monomer", "B_monomer"])
    )
    polymers.append(polymer)

### Define Uniform Density Field

Here we define an unconfined field.

In [10]:
x_width = 25
y_width = x_width
z_width = x_width

n_bins_x = x_width / (2 * interaction_length)
n_bins_y = n_bins_x
n_bins_z = n_bins_x

chi = 0

udf = UniformDensityField(
    polymers = polymers,
    binders = monomers,
    x_width = x_width,
    nx = n_bins_x,
    y_width = y_width,
    ny = n_bins_y,
    z_width = z_width,
    nz = n_bins_z,
    confine_type = "",
    confine_length = 0,
    chi = chi
)

NameError: name 'UniformDensityField' is not defined

### Specify Simulation Hyperparameters

Hyperparameters include the types of Monte Carlo moves to include, the move amplitude (maximum range of change made by a single Monte Carlo move), bead selection amplitude (maximum number of beads affected by a single Monte Carlo move), number of snapshots to run the simulation, and number of steps to include in each snapshot.

In [ ]:
move_list = [
    mv.crank_shaft,
    mv.end_pivot,
    mv.slide,
    mv.tangent_rotation,
    mv.full_chain_rotation,
    mv.full_chain_translation
]

In [ ]:
amp_bead_bounds, amp_move_bounds = mc.get_amplitude_bounds(polymers)

In [ ]:
num_snapshots = 100
mc_steps_per_snapshot = 10000

In [ ]:
latest_sim = get_unique_subfolder_name("output/sim_")
moves_to_use = ctrl.specific_moves(
    move_fxns = move_list,
    log_dir = latest_sim,
    bead_amp_bounds = amp_bead_bounds.bounds,
    move_amp_bounds = amp_move_bounds.bounds,
    controller = ctrl.SimpleControl
)

### Run the simulation

Begin generating snapshots of polymer configurations.

In [ ]:
print("Starting new simulation...")
mc.polymer_in_field(
    polymers,
    monomers,
    udf,
    mc_steps_per_snapshot,
    num_snapshots,
    amp_bead_bounds,
    amp_move_bounds,
    output_dir='output',
    mc_move_controllers=moves_to_use
)